### Imports and function definitions

In [5]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data"


import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
#print(sys.path)
import uvmodlib.v1 as uvmod

### USES Conf Templates

In [33]:
BlazarTemplate = """mesh
{  
    data  blazar ./modules/$folderName/BlazarSolidModel_v2.1.obj
    
#    cullRadius $cr
    glslVersion 330
    enableTypeCheck true
    
    propertyCollection 
    {        
        __objectName__
        {           
            vec1f cloudAlpha 0.95 | public | desc "alpha for clouds" | widget slider | range 0 1
            vec1f jetAlpha 0.7 | public | desc "alpha for jet" | widget slider | range 0 1
        }
    }

    pass 
    {
        
        useDataObject blazar
        
        shader
        {
            
            type defaultMeshShader
            {
                mtrl jetMaterial
                vertexShader   ./modules/$folderName/blazar.vs
                geometryShader ./modules/$folderName/jet.gs
                fragmentShader ./modules/$folderName/jet.fs
                
                stateManagerVar __objectName__.jetAlpha  jetAlpha
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE true
                    UV_WRITE_MASK_DEPTH false
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
                
            }            
            type defaultMeshShader
            {
                mtrl blackHoleMaterial
                vertexShader   ./modules/$folderName/blazar.vs
                geometryShader ./modules/$folderName/blazar.gs
                fragmentShader ./modules/$folderName/blazar.fs
            }

            type defaultMeshShader
            {
                mtrl accretionDiskMaterial
                vertexShader   ./modules/$folderName/blazar.vs
                geometryShader ./modules/$folderName/blazar.gs
                fragmentShader ./modules/$folderName/accretionDisk.fs
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                }  
            }

            type defaultMeshShader
            {
                mtrl exteriorDiskMaterial
                vertexShader   ./modules/$folderName/exteriorDisk.vs
                geometryShader ./modules/$folderName/blazar.gs
                fragmentShader ./modules/$folderName/exteriorDisk.fs
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                } 
            }

            type defaultMeshShader
            {
                mtrl cloudMaterial
                vertexShader   ./modules/$folderName/exteriorDisk.vs
                geometryShader ./modules/$folderName/jet.gs
                fragmentShader ./modules/$folderName/clouds.fs
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                } 
                
                stateManagerVar __objectName__.cloudAlpha  cloudAlpha
                
                glState
                {
                    UV_CULL_FACE_ENABLE true
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE true
                    UV_WRITE_MASK_DEPTH false
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }


        }
            
    }
    

    
    
}"""

### Blazar Class

In [34]:
class BlazarClass():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("BlazarSolidModel_v2.1.obj",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("blazar.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("blazar.gs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("blazar.fs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("jet.gs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("jet.fs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("accretionDisk.fs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("exteriorDisk.vs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("exteriorDisk.fs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("clouds.fs",dataFolder)        
        self.cr = 1000
    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(BlazarTemplate)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir, cr = self.cr))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [35]:
Blazar = BlazarClass(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [36]:
Blazar.object.setcameraradius(10)
Blazar.object.setcoord("SolarSystem")
Blazar.object.setname("Blazar")
Blazar.object.setguiname("/KavliLecture/Larson/Blazar")
Blazar.object.settargetradius(2000)
Blazar.object.setscalefactor(660.0)
Blazar.object.showatstartup(False)

#Blazar.cr = 10000

modinfo.setname("Blazar Model")
modinfo.setauthor("Aaron Geller<sup>1</sup> and Shane Larson<sup>2</sup><br />(1)Adler Planetarium,<br />(2)Northwestern University")
modinfo.cleardependencies()
modinfo.setdesc("A 3D model of a Blazar to accompany the IceCube discovery.")
modinfo.setversion("1.0")

generator.generate("Blazar",[],[Blazar],modinfo)
uvmod.Utility.senduvcommand(Blazar.object.name+".reload;")

## To reload

In [37]:
from config import Settings

In [46]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+Blazar.object.name)
uvmod.Utility.senduvcommand(Blazar.object.name+".reload;")

### Create colormap texture

In [454]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig("data/cmap.png", bbox_inches='tight',pad_inches=0)

plot_cmap('Blues_r')

plt.show()